In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def load_month_races(year : int, month : int) -> pd.DataFrame:
    url = f'https://cqranking.com/men/asp/gen/RaceCal.asp?year={year}&month={month}'
    race_df = pd.read_html(url, match = "Nbr")[0].dropna(axis=1, how='all')
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', {'class': 'border'})
    race_df = pd.read_html(str(table))[0].dropna(axis=1, how='all')
    race_df['race_id'] = [int(a['href'].split('=')[1]) for a in table.find_all('a') if 'raceid' in a['href']]
    return race_df

months = range(1, 13)
race_dfs = (load_month_races(2022, month) for month in months)
races = pd.concat(race_dfs)
races

In [ ]:
races_file_name = 'races.csv'

(races
 [['Start date', 'End date', 'Cat.', 'Name', 'race_id']]
 .rename(columns = {'Start date': 'start', 'End date': 'end', 'Cat.': 'cat', 'Name': 'name'})
 .to_csv(races_file_name, index = False)
)
pd.read_csv(races_file_name)